In [0]:
import requests
import json
import uuid
import os
from dotenv import load_dotenv
from pyspark.sql.functions import expr,col,udf,from_unixtime
from datetime import datetime
from pyspark.sql.types import StructType,StructField,StringType,DoubleType,IntegerType,TimestampType,FloatType,ArrayType

In [0]:
@logs
def weather_data(df):
    # Schema for the raw_weather_table
    weather_schema = StructType([
        StructField('visibility', IntegerType(), True),
        StructField('timezone', IntegerType(), True),
        StructField('main', StructType([
            StructField('temp', FloatType(), True),
            StructField('feels_like', FloatType(), True),
            StructField('temp_min', FloatType(), True),
            StructField('temp_max', FloatType(), True),
            StructField('pressure', IntegerType(), True),
            StructField('humidity', IntegerType(), True),
            StructField('sea_level', IntegerType(), True),
            StructField('grnd_level', IntegerType(), True)
        ])),
        StructField('clouds', StructType([
            StructField('all', FloatType(), True)
        ])),
        StructField('sys', StructType([
            StructField('country', StringType(), True),
            StructField('sunrise', IntegerType(), True),
            StructField('sunset', IntegerType(), True)
        ])),
        StructField('dt', IntegerType(), True),
        StructField('coord', StructType([
            StructField('lon', FloatType(), True),
            StructField('lat', FloatType(), True)
        ])),
        StructField('name', StringType(), True),
        StructField('weather', ArrayType(StructType([
            StructField('id', IntegerType(), True),
            StructField('main', StringType(), True),
            StructField('description', StringType(), True),
            StructField('icon', StringType(), True)
        ]), True)),
        StructField('cod', IntegerType(), True),
        StructField('id', IntegerType(), True),
        StructField('wind', StructType([
            StructField('speed', IntegerType(), True),
            StructField('deg', IntegerType(), True),
            StructField('gust', FloatType(), True)
        ])),
        StructField('base', StringType(), True)
    ])
    
    dbutils.fs.cp('dbfs:/FileStore/.env', "file:/tmp/.env")
    load_dotenv("/tmp/.env")    
    key = os.getenv("API")
    api_endpoint =  "https://api.openweathermap.org/data/2.5/weather"


    def fetch_data(city):
        url = f"{api_endpoint}?q={city}&appid={key}"
        response = requests.get(url)

        if response.status_code == 200:
            return response.json()
        else:
            return None

    fetch_weather_udf = udf(lambda x: fetch_data(x), weather_schema)

    df = df.withColumn("weather", fetch_weather_udf(col("name"))).select("weather")

    start = datetime.fromtimestamp(df.selectExpr("min(weather.dt)").first()[0])
    end = datetime.fromtimestamp(df.selectExpr("max(weather.dt)").first()[0])

    return df, start, end


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-516872996639980>:1
----> 1 @logs
      2 def weather_data(df):
      3     # Schema for the raw_weather_table
      4     weather_schema = StructType([
      5         StructField('visibility', IntegerType(), True),
      6         StructField('timezone', IntegerType(), True),
   (...)
     44         StructField('base', StringType(), True)
     45     ])
     47     def fetch_data(city):

NameError: name 'logs' is not defined